In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )


In [ ]:
PP = 10
dp_ss = dsdat[PP]['dp_ss']
fobs = data[PP]['fobs_cents']

In [ ]:
print(dp_ss.shape)
print(np.unravel_index(np.argmax(dp_ss), shape=dp_ss.shape))

### find a realization that can have high dp_ss

In [ ]:
RR, ss = np.unravel_index(np.argmax(dp_ss), shape=dp_ss.shape)
print(f"{RR=}, {ss=}, {dp_ss[RR,ss]=}")

check the individual gamm_ssi

In [ ]:
gamma_ssi = dsdat[PP]['gamma_ssi'][:,RR,ss]
ev_ss = dsdat[PP]['ev_ss'][RR]
print(gamma_ssi.shape)
fig, ax = plot.figax(xlabel='fobs', ylabel='DP_i')
ax.scatter(np.repeat(fobs, gamma_ssi[0].size).reshape(gamma_ssi.shape), gamma_ssi)


check other sky realizations, many have high dp_ss. nice

In [ ]:
print(np.sum(dp_ss[RR]))
fig, ax = plot.figax(xlabel='sky realizations', ylabel='dp_ss', xscale='linear')
ax.scatter(np.arange(len(dp_ss[RR])), dp_ss[RR], alpha=0.5)
fig
fig, ax = plot.figax(xlabel='sky realizations', ylabel='ev_ss', xscale='linear')
ax.scatter(np.arange(len(ev_ss)), ev_ss, alpha=0.5, color='orange')

use the hc_ss and hc_bg from this realization

In [ ]:
fig, ax = plot.figax(xlabel='fobs', ylabel='hc')
hc_bg, hc_ss = data[PP]['hc_bg'][:,RR], data[PP]['hc_ss'][:,RR]
ax.plot(fobs, hc_bg)
ax.scatter(np.repeat(fobs, hc_ss[0].size).reshape(hc_ss.shape), hc_ss,
           alpha=0.25)

# make a pta for fresh detstats

In [ ]:
seed = 10000
if seed is None:
    seed = np.random.randint(99999) # get a random number
print(f"{seed=}")
np.random.seed(seed)

psrs, red_amp, sigma, _, _ = detstats.calibrate_one_pta(hc_bg, hc_ss, fobs, npsrs=40, seed=seed,
                                                        ret_sig=True)

print(red_amp, sigma)

# check distinction/matching w random seed
max_phi, max_theta = 0, 0
am_phi, am_theta = None, None
for ii, psr in enumerate(psrs):
    if psr.phi > max_phi:
        max_phi = psr.phi
        am_phi = ii
    if psr.theta > max_theta:
        max_theta = psr.theta
        am_theta = ii

print(f"{am_theta=}, {am_phi=}")

In [ ]:
_dp_bg, _snr_bg = detstats.detect_bg_pta(psrs, fobs, hc_bg[:,np.newaxis], ret_snr=True)
print(f"{_dp_bg=}, {_snr_bg=}")

# get single source detstats
_dp_ss, _snr_ss, _gamma_ssi = detstats.detect_ss_pta(
        psrs, fobs, hc_ss[:,np.newaxis], hc_bg[:,np.newaxis], 
        nskies=1, ret_snr=True,)
print(f"{np.sum(_gamma_ssi)=}")
print(f"{detstats.expval_of_ss(_gamma_ssi)=}")
print(f"{detstats.count_n_ss(_gamma_ssi)=}")

In [ ]:
print(dsdat[0].keys())
print(dsdat[PP]['sigmas'].shape)
sigma = np.mean(dsdat[PP]['sigmas'])
print(f"{sigma=}, {np.mean(dsdat[PP]['sigmas'])=}")

In [ ]:
hc_bg, hc_ss = data[PP]['hc_bg'][:,RR], data[PP]['hc_ss'][:,RR]
hc_ss[0,1] = hc_ss[0,0]
fig, ax = plot.figax(xlabel='fobs', ylabel='hc')
ax.plot(fobs, hc_bg)
ax.scatter(np.repeat(fobs, hc_ss[0].size).reshape(hc_ss.shape), hc_ss,
           alpha=0.25)



In [ ]:
plot.plot_bg_ss(fobs, )